In [1]:
#pip install selenium
#pip install pandas
#pip install openpyxl
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
import datetime
import re
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
 

 

# Create a new instance of the Firefox driver
from selenium import webdriver

 

# Firefox tarayıcının yolunu belirtin
#firefox_path = "C:\\Users\\necati.keskin\\AppData\\Local\\Microsoft\\WindowsApps\\firefox.exe"

 

# Firefox WebDriver'ı başlatın
#driver = webdriver.Firefox(firefox_binary=firefox_path)

 

# Navigate to the webpage
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get("https://www.binance.com/en/simple-earn")



# Wait for the page to load
driver.implicitly_wait(10)
#### Closing pop-ups
time.sleep(3)
target_element = driver.find_element(By.XPATH, '//*[@id="onetrust-reject-all-handler"]')
target_element.click()
time.sleep(1)
target_element = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[3]/button')
target_element.click()
######


####
time.sleep(2)
target_element = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[4]/div[2]')
text = target_element.text
match = re.search(r'\d+', text)
if match:
    different_type_of_earn_assets = int(match.group())
    print("Extracted number for total Earn Products:", different_type_of_earn_assets)
else:
    print("No number found in the string")


target_element.click()
#########





#### Checking for body_div number

html_body_div_list=[9,6,11]
for i in html_body_div_list:
    try:
        element=driver.find_element(By.XPATH, '/html/body/div['+str(i)+']/div[1]/div/div[2]/div/div/div[1]')
        # If element is found, do something with it here
        print("Element found for i=:"+str(i))
        print(f'html_body_div[{i}]')
        html_body_div = i
        break
    except:
        print("Element not found or error occurred in i="+str(i))
    finally:
        print("If you can't get body div; Check for /html/body/div[?] in source !")
#####





# Start the timer
start_time = time.time()

apr_list = []
unique_assets = []
for i in range(1, different_type_of_earn_assets+6):
    try:
        asset_xpath = f'/html/body/div[{html_body_div}]/div[1]/div/div[2]/div/div/div[{str(i)}]'
        target_element = driver.find_element(By.XPATH, asset_xpath)
        target_element.location_once_scrolled_into_view #new
        text = target_element.text
        lines = text.split('\n')

        # Find the index of 'Duration(Days)'
        duration_index = -1

        for i, line in enumerate(lines):
            if line == 'Duration(Days)':
                duration_index = i
                break
        first_item = lines[0]
        if first_item in unique_assets:
            pass
        else:
            unique_assets.append(first_item)
            # Extract the duration values
            duration_values = []
            if duration_index != -1:
                for i in range(duration_index + 1, len(lines)):
                    if re.match(r'^\d+$', lines[i]) or lines[i] == 'Flexible':
                        duration_values.append(lines[i])    

            try:

                target_element = driver.find_element(By.XPATH, asset_xpath)
                target_element.location_once_scrolled_into_view #new 

                # ... Your other code ...

                for j in range(1, len(duration_values)+1):
                    asset_future_xpath = asset_xpath + f'/div[2]/div[1]/div[2]/div[{j}]'
                    try:
                        asset = []
                        target_element = driver.find_element(By.XPATH, asset_future_xpath)
                        #target_element.location_once_scrolled_into_view
                        target_element.click()
                        asset.append(target_element.text)  # getting future text like flex or 30
                        ## Extracting Text
                        target_element = driver.find_element(By.XPATH, asset_xpath)
                        lines = target_element.text.split('\n')
                        asset.append(lines[0].split('\\')[0])
                        if '+' in lines:
                            asset.append((lines[1].split('\\')[0])+(lines[2].split('\\')[0])+(lines[3].split('\\')[0]))
                        else:
                            asset.append(lines[1].split('\\')[0])
                        asset.append(lines[-1].split('\\')[-1])
                        #print(asset)
                        apr_list.append(asset)
                        #print(len(apr_list))
                        print("Wait for scrape...")
                        print("*")
                        print("*")
                        print("*")
                    except Exception as e:
                        print("Inner Exception:", str(e))
                        break

            except Exception as e:
                print("Outer Exception:", str(e))


            
    except Exception as e:
        # should add another try then finally Close the browser #driver.quit()
        print("Outer Exception Last:", "Pre-calculated number of asset scrape has finished!")            
        break

# Calculate and print the execution time
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")
as_min=execution_time/60
print(f"Execution time: {as_min} minutes")

try:
    asset_xpath = f'/html/body/div[{html_body_div}]/div[1]/div/div[2]/div/div/div[{str(different_type_of_earn_assets+7)}]'
    target_element = driver.find_element(By.XPATH, asset_xpath)
    print('Check the source! Total earn asset number might be changed!')
except:
    print('All the data scraped! Enjoy your day!')
driver.quit()

# saving data
columns = ['Days', 'Asset', 'APR', 'Condition']
today = datetime.date.today()
df = pd.DataFrame(apr_list, columns=columns)
df.to_excel(f'{str(today)}-Binance-APRs.xlsx', index=False)
print(f'The data has been saved as "{str(today)}-Binance-APRs.xlsx"')

#
##### Future notes:
#while 
#    target_element = driver.find_element(By.XPATH, '/html/body/div[9]/div[1]/div/div[2]/div[2]')
 #   target_element.text
  #  no element to be list

Extracted number for total Earn Products: 356
Element not found or error occurred in i=9
If you can't get body div; Check for /html/body/div[?] in source !
Element found for i=:6
html_body_div[6]
If you can't get body div; Check for /html/body/div[?] in source !
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scrape...
*
*
*
Wait for scr

In [22]:
columns = ['Days', 'Asset', 'APR', 'Condition']
today = datetime.date.today()
df = pd.DataFrame(apr_list, columns=columns)
save_path = f"C:\\Users\\suat.tuncer\\OneDrive - ICRYPEX BILISIM A.S\\Desktop\\APRs\\{today}-Binance-APRs.xlsx"
df.to_excel(save_path, index=False)
print(f'The data has been saved as "{str(today)}-Binance-APRs.xlsx"')

The data has been saved as "2023-09-06-Binance-APRs.xlsx"


In [21]:
df.head()

,Days,Asset,APR,Condition
0,Flexible,FDUSD,1.01%+6%,Subscribe
1,Flexible,BLZ,28.61%,Subscribe
2,Flexible,CTK,15.57%,Subscribe
3,30,CTK,11.9%,Subscribe
4,60,CTK,12.9%,Subscribe
